In [262]:
CHARSET = "<>[]+-.,"
ERROR = "Error!"
REDUNDANCIES = ["+-", "-+", "<>", "><", "[]"]

def strip_chars(stdio):
    global CHARSET
    return ''.join([c for c in stdio if c in CHARSET])


def remove_redundant(stdio):
    global REDUNDANCIES
    output = stdio
    for redundancy in REDUNDANCIES:
        output = output.replace(redundancy, '')
    output2 = output
    for redundancy in REDUNDANCIES:
        output2 = output2.replace(redundancy, '')
    if output != output2:
        return remove_redundant(output2)
    else:
        return output2


def tokenize(stdio):
    ##  "++++--->>>[++]"
    ##  [ ('+', 1), ('>', 3), ('[', tokenize("++")) ]) ]
    ## [ ('+', 1), ('>', 3), ('[', [('+', 2)]) ]) ]
    expr = [] 
    i = 0
    while i < len(stdio):
        # Set up lastchar and char vars
        char = stdio[i]
        if i >= 1:
            lastchar = stdio[i-1]
        else:
            lastchar = ""
    
        # Actual tokenization done here
        if char in "+-":
            if (lastchar != "") and (lastchar in "+-"):
                # Add or subtract 1 from last token
                expr[-1][1] += eval(f"{char}1") # "+1" or "-1"
                i += 1
            else:
                token = ['+', (-1,1)[char=="+"]] #('+', 1) if char=="+" else -1
                expr.append(token)
                i += 1
        # Same concept as "+-" charset
        if char in "><":
            val = (-1,1)[char==">"]
            if (lastchar != "") and (lastchar in "><"):
                expr[-1][1] += eval(f"{val}")
                i += 1
            else:
                token = ['>', (-1,1)[char==">"]]
                expr.append(token)
                i += 1
        # No argument, so token is an str and not a list
        if char in '.,':
            token = char
            expr.append(token)
            i += 1   
        # Bracket tokenization here
        if char == "]":
            return ERROR
        if char == "[":
            i += 1
            depth = 1
            temp_expr = ''
            while depth != 0:
                try:
                    char = stdio[i]
                except:
                    return ERROR
                if char == ']' and depth == 0:
                    break
                if char == '[':
                    depth += 1
                if char == ']':
                    depth -= 1
                if depth == 0:
                    break
                temp_expr += char
                i += 1
            token = ['[',tokenize(temp_expr)]
            expr.append(token)
            i += 1 # skips the initial '[' and also
                   # the ']' that is the current char
    return expr


def translate(tokens):
    tabulate = lambda line: " "*2 + line
    char_tokens = {
        "." : "putchar(*p);",
        "," : "*p = getchar();",
    }
    output_lines = []
    for token in tokens:
        if type(token) == str:
            output_lines.append(char_tokens[token])
        if token[0] == '>':
            arg = token[1]
            sign = ('+','-')[arg<=0]
            output_lines.append(f"p {sign}= {abs(arg)};")
        if token[0] == '+':
            arg = token[1]
            sign = ('+','-')[arg<=0]
            output_lines.append(f"*p {sign}= {abs(arg)};")
        if token[0] == '[':
            arg = token[1]
            loop_lines = ["if (*p) do {"] +\
                         list(map(tabulate, translate(arg))) +\
                         ["} while (*p);"]
            output_lines += loop_lines
    return output_lines
                
    
def brainfuck_to_c(bf):
    formatted_bf = remove_redundant(strip_chars(bf))
    tokens = tokenize(formatted_bf)
    if tokens == ERROR:
        return ERROR
    translated_bf = translate(tokens)
    compiled = '\n'.join(translated_bf)
    return compiled + ('\n',"")[compiled==""]

In [265]:
# Example
bf = "[[.]+++]"
print(brainfuck_to_c(bf))

if (*p) do {
  if (*p) do {
    putchar(*p);
  } while (*p);
  *p += 3;
} while (*p);

